In [1]:
import pysam
import os
import pandas as pd
import numpy as np
import time
import argparse
import sys
import pybedtools


In [2]:
#This script calculates the frequency of each GC content for fragments that overlap the non-blacklisted areas
#This is performed for each fragment size in the range specified
#this only needs to be performed once for each filter


In [3]:
# %matplotlib inline
# #arguments for testing 
# mappable_regions_path = '/fh/fast/ha_g/user/adoebley/projects/griffin_revisions_1/genome/k100_minus_exclusion_lists.mappable_regions.hg38.bed'

# ref_seq_path = '/fh/fast/ha_g/grp/reference/GRCh38/GRCh38.fa'
# chrom_sizes_path = '/fh/fast/ha_g/grp/reference/GRCh38/hg38.standard.chrom.sizes'
# out_dir = 'tmp'

# fragment_length = 165 #fragment length
# read_length = 100

In [4]:
parser = argparse.ArgumentParser()

parser.add_argument('--mappable_regions_path', help='highly mappable regions to be used in GC correction, bed or bedGraph format', required=True)
parser.add_argument('--ref_seq',help='reference sequence (fasta format)',required=True)
parser.add_argument('--chrom_sizes',help='path to chromosome sizes for the reference seq',required=True)
parser.add_argument('--out_dir',help='folder for results',required=True)
parser.add_argument('--fragment_length',help='length of fragment (in bp) for which GC will be calculated',type=int, required=True)
parser.add_argument('--read_length',help='length of read (in bp)',type=int, required=True)

args = parser.parse_args()

mappable_regions_path=args.mappable_regions_path
ref_seq_path = args.ref_seq
chrom_sizes_path = args.chrom_sizes
out_dir = args.out_dir
fragment_length = args.fragment_length
read_length = args.read_length


In [5]:
mappable_name = mappable_regions_path.rsplit('/',1)[1].rsplit('.',1)[0]
out_file = out_dir+'/'+mappable_name+'.'+str(fragment_length)+'bp.GC_frequency.txt'

In [6]:
#keep autosomes only
chroms = ['chr'+str(m) for m in range(1,23)]

In [7]:
if read_length>fragment_length:
    read_length = fragment_length 

In [8]:
print('mappable_regions_path:',mappable_regions_path)
print('out_file:',out_file)
print('fragment_length:',fragment_length)
print('read_length:',read_length)

mappable_regions_path: /fh/fast/ha_g/user/adoebley/projects/nucleosome_profiling_griffin/add_mappability_1/genome_info/k100_exclusion_lists.mappable_regions.bed
out_file: tmp/k100_exclusion_lists.mappable_regions.165bp.GC_frequency.txt
fragment_length: 165
read_length: 100


In [9]:
#import filter
mappable_intervals = pd.read_csv(mappable_regions_path, sep='\t', header=None)

mappable_intervals = mappable_intervals[mappable_intervals[0].isin(chroms)]

print('chroms:', chroms)
print('number_of_intervals:',len(mappable_intervals))
sys.stdout.flush()

chroms: ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22']
number_of_intervals: 467958


In [10]:
print(mappable_intervals.head())

      0      1      2
0  chr1  10107  10439
1  chr1  10581  10622
2  chr1  13325  13326
3  chr1  13978  13979
4  chr1  15242  15273


In [11]:
#get chrom sizes info
chrom_sizes = pd.read_csv(chrom_sizes_path, sep='\t', header=None)

#also keep as a dict
chrom_size_dict = chrom_sizes.set_index(0).to_dict()[1]

In [12]:
#import the ref_seq
ref_seq=pysam.FastaFile(ref_seq_path)

In [ ]:
#count the GC content of all fragments where the forward read overlaps the specified regions
start_time = time.time()
print('Calculating forward read frequency')

#create the GC frequencies dict
fw_GC_dict = {}
for num_GC in range(0,fragment_length+1):
    fw_GC_dict[num_GC]=0
    
for i in range(len(mappable_intervals)):
    chrom = mappable_intervals.iloc[i][0]
    start = mappable_intervals.iloc[i][1]+1
    end = mappable_intervals.iloc[i][2]-1
    if i%5000==0:
        print('interval',i,':',chrom,start,end,'seconds:',np.round(time.time()-start_time))
        sys.stdout.flush()
    
    #count up all possible fw reads that overlap the interval
    #adjust the start and end so it includes all possible fragment that overlap the interval 
    adjusted_start = start-read_length
    adjusted_end = end+fragment_length
    
    if adjusted_start<0:
        adjusted_start = 0
    if adjusted_end>chrom_size_dict[chrom]:
        adjusted_end = chrom_sizes_dict[chrom]
        print(chrom,chrom_sizes_dict[chrom],'modifying_end_to_end_of_chromosome')

    #print('fetch start',adjusted_start-start)
    #print('fetch end',adjusted_end-end)
    
    fetched = ref_seq.fetch(chrom,adjusted_start,adjusted_end)
    fetched = fetched.replace('g','G').replace('c','C').replace('a','A').replace('t','T').replace('n','N')
    fetched = np.array(list(fetched.replace('G','1').replace('C','1').replace('A','0').replace('T','0').replace('N','2')),dtype=float)

    #swap the 2 for a random 1 or 0 #there has to be a better way to do this but I can't figure it out
    #the 0 or 1 is required because the sliding window sum algorithm only does integers
    #unknown nucleotides should be quite rare if the filter is done correctly
    rng = np.random.default_rng(start)
    fetched[fetched==2] = rng.integers(2, size=len(fetched[fetched==2])) #random integer in range(2) (i.e. 0 or 1)

    n=len(fetched)

    window_sum = int(sum(fetched[:fragment_length]))
    #print(k,len(fetched[:k]),window_sum)

    fw_GC_dict[window_sum]+=1
    for i in range(n-fragment_length):
        window_sum = int(window_sum - fetched[i] + fetched[i+fragment_length])
        fw_GC_dict[window_sum]+=1


interval 0 : chr1 10108 10438 seconds: 0.0
interval 5000 : chr1 27829386 27829385 seconds: 19.0
interval 10000 : chr1 67436203 67438735 seconds: 48.0
interval 15000 : chr1 99026501 99042644 seconds: 71.0
interval 20000 : chr1 120980208 120980211 seconds: 87.0
interval 25000 : chr1 146622714 146622713 seconds: 90.0
interval 30000 : chr1 152615293 152615495 seconds: 95.0
interval 35000 : chr1 184132198 184132245 seconds: 118.0
interval 40000 : chr1 221974800 221975255 seconds: 145.0
interval 45000 : chr10 4113476 4117387 seconds: 168.0
interval 50000 : chr10 42531634 42534505 seconds: 194.0
interval 55000 : chr10 50193993 50193992 seconds: 200.0
interval 60000 : chr10 80885403 80885487 seconds: 223.0
interval 65000 : chr10 114741514 114752767 seconds: 248.0
interval 70000 : chr11 14664621 14664671 seconds: 272.0
interval 75000 : chr11 49041185 49041242 seconds: 298.0
interval 80000 : chr11 71679800 71679799 seconds: 312.0
interval 85000 : chr11 94241278 94248951 seconds: 329.0
interval 9

In [ ]:
#count the GC content of all fragments where the reverse read overlaps the specified regions
print('Calculating reverse read frequency')

#create the GC frequencies dict
rv_GC_dict = {}
for num_GC in range(0,fragment_length+1):
    rv_GC_dict[num_GC]=0
    
for i in range(len(mappable_intervals)):
    chrom = mappable_intervals.iloc[i][0]
    start = mappable_intervals.iloc[i][1]+1 #skip the first and last positions because these reads aren't fetched by pysam
    end = mappable_intervals.iloc[i][2]-1
    if i%5000==0:
        print('interval',i,':',chrom,start,end,'seconds:',np.round(time.time()-start_time))
        sys.stdout.flush()
    
    #count up all possible rv reads that overlap the interval
    #adjust the start and end so it includes all possible fragment that overlap the interval 
    adjusted_start = start-fragment_length
    adjusted_end = end+read_length
    
    if adjusted_start<0:
        adjusted_start = 0
    if adjusted_end>chrom_size_dict[chrom]:
        adjusted_end = chrom_sizes_dict[chrom]
        print(chrom,chrom_sizes_dict[chrom],'modifying_end_to_end_of_chromosome')

    #print('fetch start',adjusted_start-start)
    #print('fetch end',adjusted_end-end)
        
    fetched = ref_seq.fetch(chrom,adjusted_start,adjusted_end)
    fetched = fetched.replace('g','G').replace('c','C').replace('a','A').replace('t','T').replace('n','N')
    fetched = np.array(list(fetched.replace('G','1').replace('C','1').replace('A','0').replace('T','0').replace('N','2')),dtype=float)

    #swap the 2 for a random 1 or 0 #there has to be a better way to do this but I can't figure it out
    #the 0 or 1 is required because the sliding window sum algorithm only does integers
    #unknown nucleotides should be quite rare if the filter is done correctly
    rng = np.random.default_rng(start)
    fetched[fetched==2] = rng.integers(2, size=len(fetched[fetched==2])) #random integer in range(2) (i.e. 0 or 1)

    n=len(fetched)

    window_sum = int(sum(fetched[:fragment_length]))
    #print(k,len(fetched[:k]),window_sum)

    rv_GC_dict[window_sum]+=1
    for i in range(n-fragment_length):
        window_sum = int(window_sum - fetched[i] + fetched[i+fragment_length])
        rv_GC_dict[window_sum]+=1


In [ ]:
#convert to df and export
GC_df = pd.DataFrame()
#save GC dict
current = (pd.Series(rv_GC_dict)+pd.Series(fw_GC_dict)).reset_index()
current = current.rename(columns={'index':'num_GC',0:'number_of_fragments'})
current['length']=fragment_length
current = current[['length','num_GC','number_of_fragments']]
GC_df = GC_df.append(current, ignore_index=True)
GC_df.to_csv(out_file,sep='\t',index=False)

In [ ]:
print('done')
